In [1]:
from vllm import LLM, SamplingParams
from rdflib import Graph

In [2]:
llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct", dtype="float16", max_num_batched_tokens=50_000, gpu_memory_utilization=0.01)

WARNING 07-02 17:58:40 config.py:1218] Casting torch.bfloat16 to torch.float16.
INFO 07-02 17:58:40 llm_engine.py:161] Initializing an LLM engine (v0.5.0) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


/projets/melodi/gsantoss/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-02 17:58:41 selector.py:119] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 07-02 17:58:41 selector.py:50] Using XFormers backend.
INFO 07-02 17:58:42 selector.py:119] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 07-02 17:58:42 selector.py:50] Using XFormers backend.
INFO 07-02 17:58:43 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-02 17:58:55 model_runner.py:159] Loading model weights took 14.9595 GB
INFO 07-02 17:59:05 gpu_executor.py:83] # GPU blocks: 0, # CPU blocks: 2048


ValueError: No available memory for the cache blocks. Try increasing `gpu_memory_utilization` when initializing the engine.

In [ ]:
base_ont = '/projets/melodi/gsantoss/data/oaei/tracks/complex/geolink'

o1 = Graph().parse(f'{base_ont}/rdfgmo.rdf').serialize(format='ttl')
o2 = Graph().parse(f'{base_ont}/rdfgbo.rdf').serialize(format='ttl')

txt = f'''
Given the two ontologies bellow:

<ontology1>
{o1}    
</ontology1>    
<ontology2>
{o2}
</ontology2>

And one example of alignment between two different ontologies:

<ontology1>
@prefix lib: <http://example.org/library#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

lib:Book1 a lib:Book ;
    dcterms:title "The Catcher in the Rye" ;
    dcterms:creator lib:Author1 ;
    lib:hasGenre "Fiction" .

lib:Author1 a lib:Author ;
    foaf:name "J.D. Salinger" ;
    foaf:birthDate "1919-01-01" .
</ontology1>
<ontology2>
@prefix pub: <http://example.org/publishing#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

pub:Book1 a pub:Book ;
    dcterms:title "To Kill a Mockingbird" ;
    dcterms:creator pub:Author1 ;
    pub:publicationYear "1960" .

pub:Author1 a pub:Author ;
    foaf:name "Harper Lee" ;
    pub:hasNationality "American" .
</ontology2>
<alignment>
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:align="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#"
         xmlns:edoal="http://ns.inria.org/edoal/1.0/#">

  <Alignment>
    <xml>yes</xml>
    <level>2EDOAL</level>
    <type>**</type>
    
    <onto1>
      <Ontology rdf:about="http://example.org/library#"/>
    </onto1>
    <onto2>
      <Ontology rdf:about="http://example.org/publishing#"/>
    </onto2>

    <map>
      <Cell>
        <entity1 rdf:resource="http://example.org/library#Book"/>
        <entity2 rdf:resource="http://example.org/publishing#Book"/>
        <relation>=</relation>
        <measure>1.0</measure>
      </Cell>
    </map>
    <map>
      <Cell>
        <entity1 rdf:resource="http://example.org/library#Author"/>
        <entity2 rdf:resource="http://example.org/publishing#Author"/>
        <relation>=</relation>
        <measure>1.0</measure>
      </Cell>
    </map>
  </Alignment>
</rdf:RDF>
</alignment>

Write a file in EDOAL format containing the complex alignment between the ontology1 and ontology2. You don't need to explain yourself. Just give as response the resulting file without saying anything. Here is one example bellow:
'''

In [ ]:
prompts = [
    txt,
]
sampling_params = SamplingParams(temperature=0.0, top_p=1)

In [4]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

NameError: name 'llm' is not defined